In [83]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchtext
from torchnlp.metrics import get_moses_multi_bleu
from torchtext.data import Field, BucketIterator

import tensorflow as tf
import tensorflow_datasets as tfds
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO

import linecache
import sys
import os
import re
import random
import time

## Loading the dataset

In [3]:
try:
    os.mkdir("./datasets")
except FileExistsError:
    print("Directories already exists")

# getting descriptions
!wget https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.anno -O ./datasets/all.desc

# getting code
!wget https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.code -O ./datasets/all.code

Directories already exists
--2019-10-23 08:35:57--  https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.anno
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1382085 (1.3M) [text/plain]
Saving to: './datasets/all.desc'

./datasets/all.desc 100%[===================>]   1.32M  3.70MB/s    in 0.4s    

2019-10-23 08:35:58 (3.70 MB/s) - './datasets/all.desc' saved [1382085/1382085]

--2019-10-23 08:35:59--  https://raw.githubusercontent.com/odashi/ase15-django-dataset/master/django/all.code
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 906732 (885K) [text/plain]
Saving to: './datasets/a

## Creating a token text encoder
An encoder will take a file and a splitting function and return an object able to encode and decode a string. It will also be able to save a vocab file and retrieve from file.

In [4]:
text = " append rel_to to string 'ForeignKey, (substitute the result for field_type.)"

# looks like code split need parenthesis to be matched in the same string, if not it gives an error...
def code_split(s):
    return [x.string for x in tokenize(BytesIO(s.encode('utf-8')).readline) if x.string != '' and x.string != "\n" and not x.string.isspace()][1:]

print(code_split(text))

['append', 'rel_to', 'to', 'string', "'", 'ForeignKey', ',', '(', 'substitute', 'the', 'result', 'for', 'field_type', '.', ')']


In [5]:
text = " append rel_to to string 'ForeignKey, (subs__titute the result' for field_type."

def string_split(s):
    return list(filter(lambda x: x != '' and x != "\n" and not x.isspace(), re.split('(_|\W)', s))) # this will chunk all code properly by plits strings with quotes
#     return list(filter(lambda x: x != '' and x != "\n" and not x.isspace(), re.split('(\\\'.*?\\\'|\\\".*?\\\"|_|\W)', s))) # this keeps the strings intact

print(string_split(text))

['append', 'rel', '_', 'to', 'to', 'string', "'", 'ForeignKey', ',', '(', 'subs', '_', '_', 'titute', 'the', 'result', "'", 'for', 'field', '_', 'type', '.']


## Making the input pipeline

In [6]:
def corpus_to_array(src_fp, tgt_fp):
    lines = []
    with open(src_fp, "r") as src_file, open(tgt_fp, "r") as tgt_file:
        for src, tgt in zip(src_file, tgt_file):
            lines.append((src, tgt))
    return lines

In [7]:
def filter_corpus(data, max_seq_length=200, tokenizer=string_split):
    return [(src, tgt) for src, tgt in data if len(string_split(src)) <= max_seq_length and len(string_split(tgt)) <= max_seq_length]

In [8]:
def samples_to_dataset(samples, src_field, tgt_field):
    """
    Args:
        samples: [(src_string),(tgt_string)]
        src/tgt_tokenizer: a func that takes a string and returns an array of strings
    """
    examples = []
    
    for sample in samples:
        src_string, tgt_string = sample
        examples.append(torchtext.data.Example.fromdict({"src":src_string, "tgt":tgt_string}, 
                                        fields={"src":("src",src_field), "tgt":("tgt",tgt_field)}))
        
    dataset = torchtext.data.Dataset(examples,fields={"src":src_field, "tgt":tgt_field})
    return dataset

In [9]:
data = corpus_to_array("datasets/all.desc", "datasets/all.code")
random.shuffle(data)

In [10]:
print("Max src length:", max([len(string_split(src)) for src, tgt in data]))
print("Max tgt length:", max([len(string_split(tgt)) for src, tgt in data]))

Max src length: 586
Max tgt length: 1087


In [11]:
print("Full dataset size:", len(data))
max_seq_length=200
data = filter_corpus(data, max_seq_length=200, tokenizer=string_split)
print("Limited dataset size:", len(data))

Full dataset size: 18805
Limited dataset size: 18781


In [12]:
SRC_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')
TGT_TEXT = Field(sequential=True, tokenize=string_split, init_token='<sos>',eos_token='<eos>')

dataset = samples_to_dataset(data, SRC_TEXT, TGT_TEXT)

train_dataset, val_dataset = dataset.split([0.7,0.3])

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
SRC_TEXT.build_vocab(train_dataset)
TGT_TEXT.build_vocab(train_dataset)


sample = dataset[0].src
for tok, id in zip(sample, SRC_TEXT.numericalize([sample])):
    print("{} -> {}".format(tok, id.numpy()[0]))

otherwise -> 101
if -> 15
' -> 8
\ -> 70
\ -> 70
' -> 8
is -> 11
contained -> 99
in -> 38
path -> 66
, -> 6


In [15]:
batch_size = 16

train_iterator, valid_iterator = BucketIterator.splits(
    (train_dataset, val_dataset),
    batch_size = batch_size,
    sort_key = lambda x: len(x.src)+len(x.tgt),
    device = device)

# The iterator generates batches with padded length for sequences with similar sizes, a batch is [seq_length, batch_size]

for i, batch in enumerate(train_iterator):
    print([SRC_TEXT.vocab.itos[id] for id in batch.src.cpu().numpy()[:,4]])
    break

['<sos>', 'call', 'the', 'method', 'self', '.', 'valid', '_', 'value', 'with', 'an', 'argument', 'val', ',', 'if', 'it', 'evaluates', 'to', 'false', ',', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


## Building the model


Sample transformer without positional encoding, it uses the built in transformer model

In [16]:
rand_transformer_model = nn.Transformer() # uses default hyperparameters
src = torch.rand((10, 32, 512)) # [src_seq_length, batch_size, embedding_size]
tgt = torch.rand((20, 32, 512)) # [tgt_seq_length, batch_size, embedding_size]
rand_transformer_model(src, tgt).shape # [tgt_seq_length, batch_size, embedding_size]

torch.Size([20, 32, 512])

In [17]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [18]:
class TransformerModel(nn.Module):

    def __init__(self, src_vocab_size, tgt_vocab_size, embedding_size=128, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        
        self.embedding_size = embedding_size
        self.pos_encoder = PositionalEncoding(embedding_size, dropout)
        self.src_encoder = nn.Embedding(src_vocab_size, embedding_size)
        self.tgt_encoder = nn.Embedding(tgt_vocab_size, embedding_size)
        
        self.transformer = nn.Transformer(d_model=embedding_size, nhead=1, num_encoder_layers=1, num_decoder_layers=1, dim_feedforward=256)
        self.decoder = nn.Linear(embedding_size, tgt_vocab_size)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.src_encoder.weight.data.uniform_(-initrange, initrange)
        self.tgt_encoder.weight.data.uniform_(-initrange, initrange)
        
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, tgt):
        self.tgt_mask = self._generate_square_subsequent_mask(len(tgt)).to(device)

        src = self.src_encoder(src) * math.sqrt(self.embedding_size)
        src = self.pos_encoder(src)
        
        tgt = self.tgt_encoder(tgt) * math.sqrt(self.embedding_size)
        tgt = self.pos_encoder(tgt)
        
        output = self.transformer(src, tgt, tgt_mask=self.tgt_mask)
        output = self.decoder(output)
        return output

In [19]:
src_vocab_size = len(SRC_TEXT.vocab.stoi)
tgt_vocab_size = len(TGT_TEXT.vocab.stoi)

model = TransformerModel(src_vocab_size, tgt_vocab_size).to(device) 
src = torch.randint(0, src_vocab_size, (10,8), device=device) # [src_seq_length, batch_size]
tgt = torch.randint(0, tgt_vocab_size, (20,8), device=device) # [src_seq_length, batch_size]
model(src, tgt).shape 

torch.Size([20, 8, 3963])

In [20]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    tgt_vocab_size = len(TGT_TEXT.vocab.stoi)
    for i, batch in enumerate(train_iterator):
        encoder_input = batch.src
        decoder_input = batch.tgt[:-1]
        targets = batch.tgt[1:]
        
        optimizer.zero_grad()
        output = model(encoder_input, decoder_input)
        loss = criterion(output.view(-1, tgt_vocab_size), targets.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        total_loss += loss.item()
        log_interval = 200
        if i % log_interval == 0 and i > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, i, len(train_iterator), scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [62]:
torch.zeros((2, 4), dtype=torch.int32).fill_(3).transpose(0,1)

tensor([[3, 3],
        [3, 3],
        [3, 3],
        [3, 3]], dtype=torch.int32)

In [95]:
def evaluate(eval_model, valid_iterator):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    tgt_vocab_size = len(TGT_TEXT.vocab.stoi)
    with torch.no_grad():
        results = []
        for i, batch in enumerate(valid_iterator):
            encoder_input = batch.src
            sos_id = TGT_TEXT.vocab.stoi["<sos>"]
            decoder_input = torch.zeros((1, batch.tgt.shape[1]), dtype=torch.long, device=device).fill_(sos_id)
            targets = batch.tgt[1:]
            
#             for i in range(max_seq_length):
            for i in range(5):
                output = eval_model(encoder_input, decoder_input)
                last_pred = output[-1:].argmax(dim=2)
                # top k sampling
                # indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
                # F.softmax()
                decoder_input = torch.cat((decoder_input, last_pred))
#             print(decoder_input.shape)
            for i in range(decoder_input.shape[1]):
                src_sent = " ".join([SRC_TEXT.vocab.itos[id] for id in batch.src.transpose(0,1)[i]])
                predicted_sent = " ".join([TGT_TEXT.vocab.itos[id] for id in decoder_input.transpose(0,1)[i]])
                ref_sent = " ".join([TGT_TEXT.vocab.itos[id] for id in batch.tgt.transpose(0,1)[i]])
                BLEU = get_moses_multi_bleu([predicted_sent], [ref_sent])
                result = (src_sent, ref_sent, predicted_sent, BLEU)
                print(result)
                results.append(result)
                
        return results

In [96]:
_ = evaluate(model, valid_iterator)[:3]

('<sos> try , <eos> <pad>', '<sos> try : <eos>', '<sos> c sqlsequencereset notifier NUMBER normcase', 0.0)
('<sos> try , <eos> <pad>', '<sos> try : <eos>', '<sos> c sqlsequencereset notifier NUMBER normcase', 0.0)
('<sos> return nothing . <eos>', '<sos> return <eos> <pad>', '<sos> EntitiesForbidden sum would sum would', 0.0)
('<sos> try , <eos> <pad>', '<sos> try : <eos>', '<sos> c sqlsequencereset notifier NUMBER normcase', 0.0)
('<sos> do nothing . <eos>', '<sos> pass <eos> <pad>', '<sos> unichr WidthRatioNode sum would sum', 0.0)
('<sos> try , <eos> <pad>', '<sos> try : <eos>', '<sos> c sqlsequencereset notifier NUMBER normcase', 0.0)
('<sos> do nothing . <eos>', '<sos> pass <eos> <pad>', '<sos> unichr WidthRatioNode sum would sum', 0.0)
('<sos> try , <eos> <pad>', '<sos> try : <eos>', '<sos> c sqlsequencereset notifier NUMBER normcase', 0.0)
('<sos> try , <eos> <pad>', '<sos> try : <eos>', '<sos> c sqlsequencereset notifier NUMBER normcase', 0.0)
('<sos> do nothing . <eos>', '<sos>

multi-bleu.perl script returned non-zero exit code
b'\ngzip: /tmp/tmp4eos0_gz: not in gzip format\nERROR: could not find reference file /tmp/tmp4eos0_gz at /tmp/tmp2o27kc28 line 32.\n'


('<sos> do nothing . <eos>', '<sos> pass <eos> <pad>', '<sos> unichr WidthRatioNode sum would sum', None)
('<sos> property decorator , <eos>', '<sos> @ property <eos>', '<sos> unichr sum would sum would', 0.0)
('<sos> property decorator , <eos>', '<sos> @ property <eos>', '<sos> unichr sum would sum would', 0.0)
('<sos> return result , <eos>', '<sos> return result <eos>', '<sos> EntitiesForbidden sum would sum would', 0.0)
('<sos> import bz2 . <eos>', '<sos> import bz2 <eos>', '<sos> EntitiesForbidden sum WidthRatioNode sum WidthRatioNode', 0.0)
('<sos> return code . <eos>', '<sos> return code <eos>', '<sos> EntitiesForbidden sum 1024 unichr WidthRatioNode', 0.0)
('<sos> return response . <eos>', '<sos> return response <eos>', '<sos> EntitiesForbidden sum 1024 unichr WidthRatioNode', 0.0)
('<sos> property decorator , <eos>', '<sos> @ property <eos>', '<sos> unichr sum would sum would', 0.0)
('<sos> return potfiles . <eos>', '<sos> return potfiles <eos>', '<sos> EntitiesForbidden sum 10

multi-bleu.perl script returned non-zero exit code
b'\ngzip: /tmp/tmp3_p82lgz: not in gzip format\nERROR: could not find reference file /tmp/tmp3_p82lgz at /tmp/tmpe8a4cpji line 32.\n'


('<sos> return an empty string . <eos>', "<sos> return ' ' <eos>", '<sos> unichr 3 leaf NUMBER normcase', 0.0)
('<sos> streaming is boolean True . <eos>', '<sos> streaming = True <eos>', '<sos> c sqlsequencereset rendered wordwrap lexer', None)
('<sos> substitute space for end . <eos>', '<sos> end = newline <eos>', '<sos> EntitiesForbidden sum WidthRatioNode sum WidthRatioNode', 0.0)
('<sos> if incomment is true , <eos>', '<sos> if incomment : <eos>', '<sos> unichr WidthRatioNode sum WidthRatioNode sum', 0.0)
('<sos> if message is true , <eos>', '<sos> if message : <eos>', '<sos> c sqlsequencereset maxint RendererMixin NUMBER', 0.0)
('<sos> if m is true , <eos>', '<sos> if m : <eos>', '<sos> c sqlsequencereset maxint RendererMixin NUMBER', 0.0)
('<sos> substitute DateTimeInput for widget . <eos>', '<sos> widget = DateTimeInput <eos>', '<sos> unichr WidthRatioNode sum WidthRatioNode sum', 0.0)
('<sos> substitute cookie for c . <eos>', '<sos> c = cookie <eos>', '<sos> unichr WidthRatioNo

multi-bleu.perl script returned non-zero exit code
b'\ngzip: /tmp/tmpvqg7wmgz: not in gzip format\nERROR: could not find reference file /tmp/tmpvqg7wmgz at /tmp/tmp2njrllp8 line 32.\n'


('<sos> substitute variables for self . vars . <eos>', '<sos> self . vars = <unk> <eos> <pad>', '<sos> unichr WidthRatioNode sum WidthRatioNode sum', None)
('<sos> if bf _ errors is true , <eos>', '<sos> if bf _ errors : <eos> <pad>', '<sos> EntitiesForbidden sum WidthRatioNode sum WidthRatioNode', 0.0)
('<sos> substitute fields for self . fields . <eos>', '<sos> self . fields = fields <eos> <pad>', '<sos> unichr WidthRatioNode sum WidthRatioNode sum', 0.0)
('<sos> if response . streaming is True , <eos>', '<sos> if response . streaming : <eos> <pad>', '<sos> c sqlsequencereset rendered wordwrap WidthRatioNode', 0.0)
('<sos> if literal _ match is true , <eos>', '<sos> if literal _ match : <eos> <pad>', '<sos> EntitiesForbidden sum WidthRatioNode sum WidthRatioNode', 0.0)
('<sos> contains _ nontext is boolean False . <eos>', '<sos> contains _ nontext = False <eos> <pad>', '<sos> hashvalue extend WidthRatioNode sum WidthRatioNode', 0.0)
('<sos> if self . asvar is true , <eos>', '<sos> if

KeyboardInterrupt: 

In [47]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

In [40]:
logits = torch.tensor([0.1,0.0,0.9])
top_k = top_k_top_p_filtering(logits, top_k=2)
probabilities = F.softmax(top_k, dim=-1)
torch.multinomial(probabilities, 1)

tensor([2])

In [19]:
best_val_loss = float("inf")
epochs = 3 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    print("######## Finished Epoch {} #########".format(epoch))
#     val_loss = evaluate(model, val_data)
#     print('-' * 89)
#     print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
#           'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
#                                      val_loss, math.exp(val_loss)))
#     print('-' * 89)

#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         best_model = model

    scheduler.step()

RuntimeError: CUDA out of memory. Tried to allocate 268.00 MiB (GPU 0; 1.96 GiB total capacity; 749.39 MiB already allocated; 266.94 MiB free; 24.61 MiB cached)